## **Yüksek Düzey Programlama Dönem Proje Ödevi - Clean and Dirty**

### **Change Directory and Unzip Data**

In [ ]:
%cd /content/drive/MyDrive/YDP-Clean_and_Dirty

In [ ]:
!unzip plates.zip

### **Make classification dataset as per yolo requirement**

In [3]:
import os
import random
from shutil import copyfile

def split_data(input_folder, output_folder, split_ratio=(0.8, 0.1, 0.1)):
    """
    Splits the data into training, testing, and validation sets while maintaining folder structure.

    Parameters:
        input_folder (str): Path to the folder containing the images and subfolders.
        output_folder (str): Path to the folder where the split sets will be saved.
        split_ratio (tuple): A tuple representing the ratio for train, test, and val sets.
                            Default is (0.8, 0.1, 0.1).

    Returns:
        None
    """
    # Create output folders if they don't exist
    for split in ['train', 'test', 'val']:
        folder_path = os.path.join(output_folder, split)
        os.makedirs(folder_path, exist_ok=True)

    # Get a list of all subfolders in the input folder
    subfolders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))]

    # For each subfolder, split its contents into train, test, and val
    for subfolder in subfolders:
        subfolder_path = os.path.join(input_folder, subfolder)
        output_subfolder_path_train = os.path.join(output_folder, 'train', subfolder)
        output_subfolder_path_test = os.path.join(output_folder, 'test', subfolder)
        output_subfolder_path_val = os.path.join(output_folder, 'val', subfolder)

        os.makedirs(output_subfolder_path_train, exist_ok=True)
        os.makedirs(output_subfolder_path_test, exist_ok=True)
        os.makedirs(output_subfolder_path_val, exist_ok=True)

        # Get a list of all image files in the subfolder
        image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        total_images = len(image_files)

        # Calculate the number of images for each split
        train_size = int(split_ratio[0] * total_images)
        test_size = int(split_ratio[1] * total_images)

        # Shuffle the list of image files
        random.shuffle(image_files)

        # Copy files to the respective folders
        for i, file_name in enumerate(image_files):
            source_path = os.path.join(subfolder_path, file_name)
            if i < train_size:
                destination_path = os.path.join(output_subfolder_path_train, file_name)
            elif i < train_size + test_size:
                destination_path = os.path.join(output_subfolder_path_test, file_name)
            else:
                destination_path = os.path.join(output_subfolder_path_val, file_name)

            #print(f"Copying {source_path} to {destination_path}")

            try:
                copyfile(source_path, destination_path)
            except Exception as e:
                print(f"Error copying file: {e}")

# Example usage:
input_folder = r"/content/drive/MyDrive/YDP-Clean_and_Dirty/plates/train"
output_folder = r'/content/drive/MyDrive/YDP-Clean_and_Dirty/working'
split_data(input_folder, output_folder)

### **Install and Import ultralytics**

In [4]:
# !python --version
!pip install ultralytics
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.16 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.2/112.6 GB disk)


In [5]:
from ultralytics import YOLO

from IPython.display import display, Image

### **Run training**

In [6]:
!yolo task=classify mode=train model=yolov8l-cls.pt data='/content/drive/MyDrive/YDP-Clean_and_Dirty/working' epochs=100 imgsz=256


100% 71.7M/71.7M [00:00<00:00, 115MB/s]
Ultralytics 8.3.16 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/content/drive/MyDrive/YDP-Clean_and_Dirty/working, epochs=100, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_co

### **Run Validation**

In [9]:
!yolo task=classify mode=val model=/content/drive/MyDrive/YDP-Clean_and_Dirty/runs/classify/train/weights/best.pt data='/content/drive/MyDrive/YDP-Clean_and_Dirty/working'

Ultralytics 8.3.16 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36,187,138 parameters, 0 gradients, 98.7 GFLOPs
train: /content/drive/MyDrive/YDP-Clean_and_Dirty/working/train... found 32 images in 2 classes ✅ 
val: /content/drive/MyDrive/YDP-Clean_and_Dirty/working/val... found 4 images in 2 classes ✅ 
test: /content/drive/MyDrive/YDP-Clean_and_Dirty/working/test... found 4 images in 2 classes ✅ 
val: Scanning /content/drive/MyDrive/YDP-Clean_and_Dirty/working/val... 4 images, 0 corrupt: 100% 4/4 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100% 1/1 [00:00<00:00,  6.02it/s]
                   all       0.75          1
Speed: 0.2ms preprocess, 16.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val
💡 Learn more at https://docs.ultralytics.com/modes/val


### **Run inference on Test**

In [10]:
from ultralytics import YOLO
import pandas as pd
# Load a model
model = YOLO("/content/drive/MyDrive/YDP-Clean_and_Dirty/runs/classify/train/weights/best.pt")  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model('/content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test')  # return a list of Results objects

lab = {"id":[],"label":[]}

# Process results list
for result in results:
    probs = result.probs  # Probs object for classification outputs
    print(result.path.split("/")[-1].split(".")[0],result.names[probs.top1])
    lab["id"].append(result.path.split("/")[-1].split(".")[0])
    lab["label"].append(result.names[probs.top1])
df = pd.DataFrame(lab)


image 1/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0000.jpg: 256x256 cleaned 0.52, dirty 0.48, 11.0ms
image 2/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0001.jpg: 256x256 dirty 0.61, cleaned 0.39, 11.5ms
image 3/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0002.jpg: 256x256 dirty 0.61, cleaned 0.39, 11.5ms
image 4/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0003.jpg: 256x256 dirty 0.55, cleaned 0.45, 11.1ms
image 5/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0004.jpg: 256x256 dirty 0.60, cleaned 0.40, 11.1ms
image 6/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0005.jpg: 256x256 cleaned 0.61, dirty 0.39, 11.3ms
image 7/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0006.jpg: 256x256 cleaned 0.51, dirty 0.49, 11.0ms
image 8/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/plates/test/0007.jpg: 256x256 dirty 0.55, cleaned 0.45, 13.0ms
image 9/744 /content/drive/MyDrive/YDP-Clean_and_Dirty/